In [2]:
from flask import Flask, redirect, jsonify, url_for, request
import base64
import io
from PIL import Image
from flask_cors import CORS
from werkzeug.serving import run_simple
import torch, time
import pymysql

app = Flask(__name__)


CORS(app)

@app.route("/sendFrame", methods=['POST']) 
def re():
    print(request.method)
    if request.method == 'POST':
        
        #안드로이드에서 'image'변수에 base64로 변환된 bitmap이미지
        one_data = request.form['image']
        
        #base64로 인코딩된 이미지 데이터를 디코딩하여 byte형태로 변환
        imgdata = base64.b64decode(one_data)
        
        #byte형태의 이미지 데이터를 이미지로 변환
        image = Image.open(io.BytesIO(imgdata))
        
        #이미지 사이즈 조정
        #img_resize = image.resize((int(image.width / 2), int(image.height / 2)))
        
        #display(img_resize)
        
        model = torch.hub.load('ultralytics/yolov5', 'custom', path='./best.pt', force_reload=True)
        #anthImg = 
        #이미지 사이즈 변환 필요
        image.thumbnail((416,416))

        result = model(image)
        resultpd = result.pandas().xyxy[0]
        cls = f'{resultpd.iloc[0,5]}'
        conf = round(resultpd.iloc[0,4]*100,3)
        date = time.strftime('%Y-%m-%d')
        
        # conf = int(conf)
        # conf_r = round(conf, 3)  
        # print(conf_r)  
        print(conf)
              
        
        # 질병명
        disNm = ''
        if cls in ['0', '1', '2', '3']:
            disNm = '정상'
        elif cls == '4':
            disNm = '탄저병'
        elif cls == '5':
            disNm = '갈색무늬병'
        elif cls == '6':
            disNm = '과수화상병'
        elif cls == '9':
            disNm = '점무늬낙엽병'
        elif cls == '10':
            disNm = '부란병'
        
    
        # print(request.form['id'])
        # print(disNm)
        # print(conf)
        # jsonify({"result : {cls},{conf}"}.__format__) 
        
        result_return = (disNm, request.form['id'],'null',conf,date)
        print(result_return)
        
        selfdb = pymysql.connect(host='project-db-stu.ddns.net',
                                  user='applecareplus',
                                  password='1234',
                                  db = "applecareplus",
                                  port = 3307,
                                  charset='utf8')
        selfdb.cursor = selfdb.cursor(pymysql.cursors.DictCursor)
        selfdb.cursor.execute("insert into ins_result(dis_name, user_id, ins_poto, ins_percent, ins_time) values (%s,%s,%s,%s,%s)", result_return)
        selfdb.commit()
        
        
    #결과값 리턴    
    return str('true')
    
if __name__ == "__main__":
    run_simple('172.30.1.30', 5000, app)

 * Running on http://172.30.1.30:5000/ (Press CTRL+C to quit)


POST


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\smhrd/.cache\torch\hub\master.zip
YOLOv5  2022-6-23 Python-3.7.13 torch-1.11.0 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7042489 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


58.174
('정상', 'apple_1', 'null', 58.174, '2022-06-23')


172.30.1.30 - - [23/Jun/2022 11:22:55] "POST /sendFrame HTTP/1.1" 200 -
172.30.1.30 - - [23/Jun/2022 11:22:55] "POST /sendFrame HTTP/1.1" 200 -
Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\smhrd/.cache\torch\hub\master.zip


...

172.30.1.30 - - [23/Jun/2022 11:22:56] "POST /sendFrame HTTP/1.1" 200 -
172.30.1.30 - - [23/Jun/2022 11:22:56] "POST /sendFrame HTTP/1.1" 200 -


POST


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\smhrd/.cache\torch\hub\master.zip
YOLOv5  2022-6-23 Python-3.7.13 torch-1.11.0 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7042489 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 


74.157
('탄저병', 'apple_1', 'null', 74.157, '2022-06-23')


172.30.1.30 - - [23/Jun/2022 11:23:10] "POST /sendFrame HTTP/1.1" 200 -
172.30.1.30 - - [23/Jun/2022 11:23:10] "POST /sendFrame HTTP/1.1" 200 -


In [13]:
from flask import Flask, redirect, request, Response
import base64
import io
from PIL import Image
from flask_cors import CORS
from werkzeug.serving import run_simple
import time
import os
from pathlib import Path
from glob import glob
from werkzeug.utils import secure_filename

app = Flask(__name__)
CORS(app)

@app.route("/sendFrame", methods=['POST']) 
def re():
    print(request.method)
    if request.method == 'POST':
        
        crnt_dir = os.getcwd()        
        if crnt_dir == 'c:\\Users\\smhrd\\Desktop\\model_test2\\yolov5':
            os.chdir('../')
        
        #안드로이드에서 'image'변수에 base64로 변환된 bitmap이미지
        one_data = request.form['image']
        #base64로 인코딩된 이미지 데이터를 디코딩하여 byte형태로 변환
        imgdata = base64.b64decode(one_data)
        #byte형태의 이미지 데이터를 이미지로 변환
        image = Image.open(io.BytesIO(imgdata))
        current_time = time.strftime('%y-%m-%d %H:%M:%S')
        # image.save('./images/'+ secure_filename("{}.jpg".format(current_time)), 'jpeg')
        image.save('./images/image.jpg', 'jpeg')
        # img_path = './images/', secure_filename("{}.jpg".format(current_time))
        
        crnt_dir = os.getcwd()        
        if crnt_dir == 'c:\\Users\\smhrd\\Desktop\\model_test2':
            os.chdir('./yolov5')
        
        
        
        # #이미지 분석관련 코드 작성
        
        !python detect.py --weights ./models/best.pt --img 416 --conf 0.5 --source ../images/image.jpg --save-txt --save-conf
        
        obj_class = ''
        obj_conf = ''        
        img_path = '../images/image.jpg'
        
        imgNm = Path(img_path).stem
        result_path = glob(f'./runs/detect/exp/{imgNm}.txt')

        with open(result_path[0], "r") as f:
            text = f.readlines()
            obj_class = text[0][0]
            obj_conf = f'{round(float(text[0][2:]), 2)*100}%'
        
        result_img = Image.open('./runs/detect/exp/image.jpg')
        
        # 결과값 보여주기
        print(obj_class)
        print(obj_conf)
        result_img.show()
        
        # 결과값 리턴 형식으로 바꿔주기!!  ex) 이미지 String 형식으로 바꾸기 등
        
        
        
    #결과값 리턴    
    return 'true'

if __name__ == "__main__":
    run_simple('172.30.1.30', 5000, app)

 * Running on http://172.30.1.30:5000/ (Press CTRL+C to quit)
 * Running on http://172.30.1.30:5000/ (Press CTRL+C to quit)


POST
^C
6
54.0%


detect: weights=['./models/best.pt'], source=../images/image.jpg, data=data\coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.1-243-g7cef03d Python-3.7.13 torch-1.11.0 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7042489 parameters, 0 gradients, 15.9 GFLOPs
image 1/1 C:\Users\smhrd\Desktop\model_test2\images\image.jpg: 256x416 1 9, Done. (0.020s)
Speed: 2.0ms pre-process, 19.7ms inference, 44.0ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs\detect\exp
0 labels saved to runs\detect\exp\labels


asdfsdf
9 0.9027354121208191


172.30.1.30 - - [21/Jun/2022 15:00:55] "POST /sendFrame HTTP/1.1" 200 -
172.30.1.30 - - [21/Jun/2022 15:00:55] "POST /sendFrame HTTP/1.1" 200 -


POST
asdfsdf
9 0.9027354121208191
6
54.0%


detect: weights=['./models/best.pt'], source=../images/image.jpg, data=data\coco128.yaml, imgsz=[416, 416], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.1-243-g7cef03d Python-3.7.13 torch-1.11.0 CUDA:0 (NVIDIA GeForce RTX 2070, 8192MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7042489 parameters, 0 gradients, 15.9 GFLOPs
image 1/1 C:\Users\smhrd\Desktop\model_test2\images\image.jpg: 256x416 1 9, Done. (0.015s)
Speed: 0.0ms pre-process, 15.0ms inference, 5.0ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs\detect\exp
0 labels saved to runs\detect\exp\labels
172.30.1.30 - - [21/Jun/2022 15:01:04] "POST /sendFrame HTTP/1.1" 200 -
172.30.1.30 - - [21/Jun/2022 15:01:04] "P

POST
